In [128]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import json
from selenium import webdriver
from selenium.common.exceptions import WebDriverException

In [247]:
countries = pd.read_csv('countries.csv')['code'].tolist()
df = pd.read_csv('hs_info - Copy.csv', dtype={'id': str, 'code' : str,'parent_id': str})
# df['tariff_crawled'] = ''
df

,Unnamed: 0,id,code,description,hasChildren,section,index,crawled,country,parent_id,error_message,selectable,intervalMin,intervalMax,tariff_crawled,tariff_crawled_error_message
0,0,-1,NaN,Live animals; animal products,True,"{'description': 'SECTION I', 'chapterFrom': '0...",0.0,Done,AT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,-2,NaN,Vegetable products,True,"{'description': 'SECTION II', 'chapterFrom': '...",1.0,Done,AT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,-3,NaN,Animal or vegetable fats and oils and their cl...,True,"{'description': 'SECTION III', 'chapterFrom': ...",2.0,Done,AT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,-4,NaN,"Prepared foodstuffs; beverages, spirits and vi...",True,"{'description': 'SECTION IV', 'chapterFrom': '...",3.0,Done,AT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,-5,NaN,Mineral products,True,"{'description': 'SECTION V', 'chapterFrom': '2...",4.0,Done,AT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27313,27313,5804,41090,Other,False,NaN,NaN,NaN,BE,1904,NaN,True,NaN,NaN,NaN,NaN
27314,27314,2305,50210,"Pigs', hogs' or boars' bristles and hair and w...",False,NaN,NaN,NaN,BE,505,NaN,True,NaN,NaN,NaN,NaN
27315,27315,2505,50290,Other,False,NaN,NaN,NaN,BE,505,NaN,True,NaN,NaN,NaN,NaN
27316,27316,2605,504000010,Casings,False,NaN,NaN,NaN,BE,1905,NaN,True,NaN,NaN,NaN,NaN


In [241]:
def fetch_tariff_info_with_retry(code,country,retries=5, delay=5):
    """Gửi request với retry khi lỗi."""
    for attempt in range(retries):
        try:
            url = f"https://trade.ec.europa.eu/access-to-markets/api/tariffs/get/{code}/VN/{country}"
            response = requests.get(url, timeout=60)
            response.encoding = 'utf-8'
            if response.status_code == 200:
                return response
        except requests.RequestException as e:
            print(f"Request error: {e} - Retrying {attempt+1}/{retries}...")
            time.sleep(delay)
    return None

In [252]:
filtered_df = df.loc[(df['country'] == 'AT') & (df['selectable'].notna()) & ((df['tariff_crawled'] == '') | pd.isna(df['tariff_crawled']) | (df['tariff_crawled'] == 'Error'))]

In [ ]:
for country in countries:
    while ((df['country'] == country) & (df['selectable'].notna()) & ((df['tariff_crawled'] == '') | pd.isna(df['tariff_crawled']) | (df['tariff_crawled'] == 'Error'))).any():
        filtered_df = df.loc[(df['country'] == country) & (df['selectable'].notna()) & ((df['tariff_crawled'] == '') | pd.isna(df['tariff_crawled']) | (df['tariff_crawled'] == 'Error'))]
        print(f'row for craw: {len(filtered_df)}')
        for i in filtered_df.index:
            # if df.loc[i, 'country'] == country and (df.loc[700, 'selectable'] == True) and (df.loc[i, 'tariff_crawled'] == '' or pd.isna(df.loc[i, 'tariff_crawled']) or df.loc[i, 'tariff_crawled'] == 'Error'):
                code = df.loc[i, 'code']
                country = df.loc[i, 'country']
                attempts = 0
                success = False
                error_occurred = False  # Biến để kiểm soát lỗi

                while attempts < 5 and not success:
                    try:
                        print(f"Fetching index: {i} code: {code}... (Attempt {attempts}/5)")
                        
                        response = fetch_tariff_info_with_retry(code,country)
                        if  response and response.status_code == 200:
                            measures = response.json()[0]['measures']
                            measures_df = []
                            for measure in measures:
                                measure_str = {key: str(value) for key, value in measure.items()}
                                measure_str['hs_info_code'] = code
                                with open('tariff_info.txt', "a", encoding='utf-8') as f:
                                    f.write(json.dumps(measure_str, ensure_ascii=False) + "\n")
                                    
                            df.loc[i, 'tariff_crawled'] = 'Done'
                            df.to_csv('hs_info - Copy.csv',index = False)

                            print(f"✅ Done fetching index: {i} code: {code}")
                            success = True
                        else:
                            raise Exception(f"Failed request, status code: {response.status_code}")
                    
                    except Exception as e:
                        attempts += 1
                        error_message = str(e)
                        print(f"❌ Error fetching index: {i} code: {code}... (Attempt {attempts}/5) - {e}")
                        if attempts == 5: 
                            error_occurred = True

                if error_occurred: 
                    df.loc[i, 'tariff_crawled'] = 'Error'
                    df.loc[i, 'tariff_crawled_error_message'] = error_message
                    df.to_csv('hs_info - Copy.csv',index = False)
                    print("⏳ Encountered multiple errors. Sleeping for 1 minutes...")
                    
                    # Mở trình duyệt để nhập captcha
                    print("🔓 Mở trình duyệt để nhập captcha...")
                    driver = webdriver.Chrome()
                    driver.get("https://trade.ec.europa.eu/access-to-markets/en/search?product=0101&origin=VN&destination=AT")  # Thay bằng URL cần mở
                    print("⌨️ Vui lòng nhập captcha, sau đó đóng trình duyệt.")

                    # Đo thời gian bắt đầu
                    start_captcha_time = time.time()
                    wait_time = 600  # 10 phút (600 giây)

                    # Chờ người dùng tự đóng trình duyệt, nếu quá 10 phút thì tự động đóng
                    while True:
                        try:
                            if time.time() - start_captcha_time > wait_time:
                                print("⏳ Quá 10 phút, tự động đóng trình duyệt.")
                                driver.quit()
                                break  # Thoát khỏi vòng lặp

                            # Kiểm tra trình duyệt còn mở không
                            driver.title  
                            
                            # Kiểm tra trang có bị treo không
                            if driver.execute_script("return document.readyState") != "complete":
                                print("⚠️ Trang web có thể bị treo, chờ thêm...")
                            
                        except WebDriverException:
                            print("✅ Trình duyệt đã được đóng.")
                            break  # Nếu trình duyệt bị đóng, thoát vòng lặp
                        
                        time.sleep(5)  # Tránh vòng lặp chạy quá nhanh, gây tải CPU
                    
                    time.sleep(10)  # Đợi 10 giây trước khi tiếp tục
                    break
            


row for craw: 16456
Fetching index: 700 code: 0501... (Attempt 0/5)
✅ Done fetching index: 700 code: 0501
Fetching index: 707 code: 0510... (Attempt 0/5)
✅ Done fetching index: 707 code: 0510
Fetching index: 740 code: 0814... (Attempt 0/5)
✅ Done fetching index: 740 code: 0814
Fetching index: 743 code: 0903... (Attempt 0/5)
✅ Done fetching index: 743 code: 0903
Fetching index: 767 code: 1109... (Attempt 0/5)
✅ Done fetching index: 767 code: 1109
Fetching index: 770 code: 1203... (Attempt 0/5)
✅ Done fetching index: 770 code: 1203
Fetching index: 791 code: 1506... (Attempt 0/5)
✅ Done fetching index: 791 code: 1506
Fetching index: 804 code: 1520... (Attempt 0/5)
✅ Done fetching index: 804 code: 1520
Fetching index: 816 code: 1801... (Attempt 0/5)
✅ Done fetching index: 816 code: 1801
Fetching index: 817 code: 1802... (Attempt 0/5)
✅ Done fetching index: 817 code: 1802
Fetching index: 819 code: 1804... (Attempt 0/5)
✅ Done fetching index: 819 code: 1804
Fetching index: 820 code: 1805... 

KeyboardInterrupt: 